In [1]:
import pandas as pd
import aiohttp
import asyncio
import nest_asyncio
from tqdm.notebook import tqdm

In [2]:
# Apply nest_asyncio to allow running asyncio.run() in Jupyter Notebook
nest_asyncio.apply()

In [3]:
# Load real estate data
real_estate_df = pd.read_csv('cleaned dataset.csv')
real_estate_df.head()

C:\Users\mistr184\AppData\Local\Temp\ipykernel_24940\1639344296.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  real_estate_df = pd.read_csv('cleaned dataset.csv')


,property_url,property_id,address,street_name,apartment,city,state,latitude,longitude,postcode,...,property_type,property_status,year_build,total_num_units,listing_age,RunDate,agency_name,agent_name,agent_phone,is_owned_by_zillow
0,https://www.zillow.com/homedetails/117-3rd-St-...,249518113,"117 3rd St, Wrangell, AK 99929",3rd St,NaN,Wrangell,AK,56.474518,-132.38600,99929,...,SINGLE_FAMILY,FOR_SALE,NaN,NaN,-1,2022-04-24 07:34:15,NaN,NaN,NaN,0
1,https://www.zillow.com/homedetails/LOT-2A-Fron...,2077729913,"LOT 2A Front St, Wrangell, AK 99929",LOT 2A Front St,NaN,Wrangell,AK,56.471638,-132.38686,99929,...,LOT,FOR_SALE,NaN,NaN,-1,2022-04-24 07:34:15,NaN,NaN,NaN,0
2,https://www.zillow.com/homedetails/335-Cassiar...,249518139,"335 Cassiar St, Wrangell, AK 99929",Cassiar St,NaN,Wrangell,AK,56.475697,-132.38905,99929,...,SINGLE_FAMILY,FOR_SALE,NaN,NaN,-1,2022-04-24 07:34:15,NaN,NaN,NaN,0
3,https://www.zillow.com/homedetails/532-5-Front...,2069574197,"532.5 Front St, Wrangell, AK 99929",5 Front St,NaN,Wrangell,AK,56.468544,-132.37965,99929,...,SINGLE_FAMILY,FOR_SALE,NaN,NaN,-1,2022-04-24 07:34:15,Anchor Properties,NaN,NaN,0
4,https://www.zillow.com/homedetails/L20-B5-Ever...,2077192418,"L20-B5 Evergreen St, Wrangell, AK 99929",-B5 Evergreen St,NaN,Wrangell,AK,56.478485,-132.39151,99929,...,LOT,FOR_SALE,NaN,NaN,-1,2022-04-24 07:34:15,Anchor Properties,NaN,NaN,0


In [4]:
# Define a function to fetch weather data from WeatherAPI.com asynchronously
async def fetch_weather_data(session, api_key, lat, lon):
    url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={lat},{lon}"
    try:
        async with session.get(url) as response:
            if response.status == 200:
                return await response.json()
            else:
                print(f"Error: Received status code {response.status} for URL: {url}")
                return None
    except aiohttp.ContentTypeError as e:
        print(f"ContentTypeError: {e} for URL: {url}")
        return None
    except Exception as e:
        print(f"Exception: {e} for URL: {url}")
        return None

In [5]:
# Function to process weather data
def process_weather_data(weather_data):
    if weather_data:
        current = weather_data.get('current', {})
        return {
            'temperature': current.get('temp_c'),
            'pressure': current.get('pressure_mb'),
            'humidity': current.get('humidity'),
            'weather_main': current.get('condition', {}).get('text'),
            'weather_description': current.get('condition', {}).get('text')
        }
    else:
        return {
            'temperature': None,
            'pressure': None,
            'humidity': None,
            'weather_main': None,
            'weather_description': None
        }

In [6]:
# Asynchronous function to fetch weather data for all properties in batches
async def fetch_all_weather_data(api_key, locations, batch_size=100):
    weather_data_list = []
    async with aiohttp.ClientSession() as session:
        for i in tqdm(range(0, len(locations), batch_size)):
            batch_locations = locations[i:i+batch_size]
            tasks = [fetch_weather_data(session, api_key, lat, lon) for lat, lon in batch_locations]
            batch_results = await asyncio.gather(*tasks)
            weather_data_list.extend(batch_results)
    return weather_data_list

In [7]:
# Prepare locations (latitude, longitude) for fetching weather data
locations = list(zip(real_estate_df['latitude'], real_estate_df['longitude']))
locations

[(56.474518, -132.386),
 (56.471638, -132.38686),
 (56.475697, -132.38905),
 (56.468544, -132.37965),
 (56.478485, -132.39151),
 (56.478485, -132.39151),
 (56.478485, -132.39151),
 (56.484615, -132.38663),
 (56.484615, -132.38663),
 (56.41022, -132.3407),
 (56.80682, -132.9618),
 (56.80623, -132.97113),
 (56.001015, -132.7938),
 (56.796387, -132.8551),
 (56.967194, -133.9265),
 (58.458855, -134.17915),
 (56.724983, -132.93016),
 (58.458855, -134.17915),
 (56.724358, -132.92993),
 (56.71966, -132.92967),
 (56.81621, -132.95499),
 (58.458855, -134.17915),
 (56.973057, -133.94089),
 (55.475147, -133.14809),
 (56.566036, -132.5942),
 (56.789413, -132.99115),
 (56.80692, -132.97368),
 (56.794308, -132.98181),
 (56.81132, -132.95866),
 (57.7775, -135.212),
 (56.806496, -132.95859),
 (56.811565, -132.95847),
 (56.784267, -132.97589),
 (56.792, -132.86862),
 (56.788815, -132.97272),
 (55.55359, -133.08878),
 (56.78083, -132.97102),
 (56.791218, -132.8683),
 (56.807224, -132.96034),
 (56.74098,

In [8]:
# Fetch weather data asynchronously
api_key = '6fce9d4fdd654cff8d2224035241306'
weather_data = asyncio.run(fetch_all_weather_data(api_key, locations))

  0%|          | 0/5292 [00:00<?, ?it/s]

Error: Received status code 502 for URL: http://api.weatherapi.com/v1/current.json?key=6fce9d4fdd654cff8d2224035241306&q=37.066963,-113.11566
Error: Received status code 502 for URL: http://api.weatherapi.com/v1/current.json?key=6fce9d4fdd654cff8d2224035241306&q=37.59813,-113.44351
Error: Received status code 502 for URL: http://api.weatherapi.com/v1/current.json?key=6fce9d4fdd654cff8d2224035241306&q=37.07818,-113.11703
Error: Received status code 502 for URL: http://api.weatherapi.com/v1/current.json?key=6fce9d4fdd654cff8d2224035241306&q=42.607147,-114.773384
Error: Received status code 502 for URL: http://api.weatherapi.com/v1/current.json?key=6fce9d4fdd654cff8d2224035241306&q=42.651157,-114.78446


In [10]:
# Process the weather data
processed_weather_data = [process_weather_data(data) for data in weather_data]

# Convert the weather data into a DataFrame
weather_df = pd.DataFrame(processed_weather_data)

# Append weather data to the real estate data
combined_df = pd.concat([real_estate_df, weather_df], axis=1)

# Save the combined data
combined_df.to_csv('combined_real_estate_weather.csv', index=False)